# Notebook for implementing ISCE2 interferometry to form Interferograms, and MintPy SBAS Time-Series analysis
This notebook will:
1. Find scenes from ASF Vertex using the asf_search library
2. Download said scenes
3. Use scenes to form interferograms using ISCE2
4. Utilize the closure phase to improve phase unwrapping
5. Create SBAS Time-Series of deformation using the closure phase corrected interferograms
6. Relate non-zero closure phase to surface vegatation and mositure changes using NDVI and EO moisture products
7. Report on erosion driver events and show relation to deformation and insar velocity from corrected SBAS time-series 

# To-DO
1. Get notebook working first with manually installed SLC images
2. Build off 'mintpy_get_isceburst.ipynb' to build interactive for retrieving SLCs

https://nbviewer.org/github/isce-framework/isce2-docs/blob/master/Notebooks/UNAVCO_2020/TOPS/topsApp.ipynb

# First, some helpful functions

In [9]:
# importing needed libraries for the entire notebook in one go
# %matplotlib inline
# %matplotlib widget
import os
from dateutil.parser import parse as parse_date
from datetime import datetime
import pandas as pd
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import eof
import xml.etree.ElementTree as ET
import glob
import subprocess
import re

In [2]:
def project_dir(proj_name, operatingsys):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data for ISCE2 and mintpy
    proj_name = str
        string that contains the name of your project

    operatingsys = str
        'linux' or 'wsl', the linux operating system you are using will impact where data will be save
    """

    if operatingsys == 'linux':
        print('Creating Project Working Directory in your "~" directory')
        work_dir = os.path.join(os.path.expanduser('~'),f'{proj_name}_InSAR')
    elif operatingsys == 'wsl':
        print('Creating Project Working Directory on your Windows Desktop')
        work_dir = os.path.join('/mnt/c/Users/clayc/Desktop',f'{proj_name}_InSAR')

    #creates file on your desktop containing the work of this notebook
    os.makedirs(work_dir, exist_ok=True)
    
    # file inside work_dir for isce2 interferometry
    if_dir = os.path.join(work_dir,'interferometry')
    os.makedirs(if_dir, exist_ok=True)
    
    # file inside work_dir for mintpy time-series
    ts_dir = os.path.join(work_dir,'time_series')
    os.makedirs(ts_dir, exist_ok=True)
    

    xmls_dir = os.path.join(if_dir,'xmls')
    os.makedirs(xmls_dir, exist_ok=True)
    
    xmldirectories=[]
    for dir in ['topsApp', 'reference', 'secondary']:
        dirpath = os.path.join(xmls_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        xmldirectories.append(dirpath)

    ifdirectories=[]
    for dir in ['reference', 'secondary', 'orbits', 'work']:
        dirpath = os.path.join(if_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        ifdirectories.append(dirpath)

    tsdirectories=[]
    for dir in ['baselines', 'reference', 'merged', 'secondaries', 'mintpy']:
        dirpath = os.path.join(ts_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        if dir == 'merged':
                geomref_dir = os.path.join(dirpath,'geom_reference')
                os.makedirs(geomref_dir, exist_ok=True)

                interfer_dir = os.path.join(dirpath,'interferograms')
                os.makedirs(interfer_dir, exist_ok=True)

                tsdirectories.append(geomref_dir)
                tsdirectories.append(interfer_dir)

        tsdirectories.append(dirpath)

    return work_dir, ifdirectories, tsdirectories, xmldirectories

In [4]:
# function to create the reference and secondary .xml files needed for ISCE2 interferometry
# function to write the 
def ref_sec_xml(slc_zips_list, slc_zips_dirs):
    for i in range(len(slc_zips_dirs)):
        for j, type in enumerate(['reference', 'secondary']):
            imset = ET.Element('component', name=type)
            safe = ET.SubElement(imset, 'property', name='safe').text = slc_zips_dirs[i]
            out_dir =ET.SubElement(imset, 'property', name='output directory').text = ifdirectories[j]
            orbit_dir =ET.SubElement(imset, 'property', name='orbit directory').text = ifdirectories[2]
            roi = ET.SubElement(imset, 'property', name='region of interest').text = str(isce_aoi)
            tree = ET.ElementTree(imset)
            tree.write(os.path.join(xmldirectories[j+1], f'{slc_zips_list[i][17:25]}{type[:3]}.xml'))

In [ ]:
def topsApp_xml(xmlname, do_iono):
    """
    Create a topsApp XML configuration file for Sentinel-1 InSAR processing.

    Parameters:
        xmlname (str): Name of the output XML file.
        do_iono (str): 'True' or 'False', to enable or disable ionospheric correction.
        isce_aoi (list): Bounding box for processing [S, N, W, E].
    """

    data = ET.Element('topsApp')
    tinsar = ET.SubElement(data, 'component', name='topsinsar')

    # Sensor and DEM settings
    ET.SubElement(tinsar, 'property', name='sensor name').text = 'SENTINEL1'
    # ET.SubElement(tinsar, 'property', name='useHighResolutionDemOnly').text = str(True)
    # ET.SubElement(tinsar, 'property', name='demFilename').text = '/path/to/dem.tif'
    # ET.SubElement(tinsar, 'property', name='geocode demfilename').text = '/path/to/geocode_dem.tif'

    # Reference and Secondary catalogs
    ref_xmlcomp = ET.SubElement(tinsar, 'component', name='reference')
    ET.SubElement(ref_xmlcomp, 'catalog').text = os.path.join(xmldirectories[1], f"{xmlname[:8]}ref.xml")
    sec_xmlcomp = ET.SubElement(tinsar, 'component', name='secondary')
    ET.SubElement(sec_xmlcomp, 'catalog').text = os.path.join(xmldirectories[2], f"{xmlname[9:17]}sec.xml")

    # General processing settings
    ET.SubElement(tinsar, 'property', name='swaths').text = str([3])
    ET.SubElement(tinsar, 'property', name='azimuth looks').text = str(1)
    ET.SubElement(tinsar, 'property', name='range looks').text = str(5)
    ET.SubElement(tinsar, 'property', name='filter strength').text = str(0.2)
    ET.SubElement(tinsar, 'property', name='geocode bounding box').text = " ".join(map(str, isce_aoi))

    # Unwrapping settings
    ET.SubElement(tinsar, 'property', name='do unwrap').text = str(True)
    ET.SubElement(tinsar, 'property', name='unwrapper name').text = 'snaphu_mcf'
    # ET.SubElement(tinsar, 'property', name='do unwrap 2 stage').text = str(False)
    # ET.SubElement(tinsar, 'property', name='unwrapper 2stage name').text = 'REDARC0'
    # ET.SubElement(tinsar, 'property', name='SOLVER_2STAGE').text = 'pulp'

    # ESD settings
    ET.SubElement(tinsar, 'property', name='do ESD').text = str(True)
    ET.SubElement(tinsar, 'property', name='ESD azimuth looks').text = str(1)
    ET.SubElement(tinsar, 'property', name='ESD range looks').text = str(5)
    ET.SubElement(tinsar, 'property', name='ESD coherence threshold').text = str(0.85)
    ET.SubElement(tinsar, 'property', name='extra ESD cycles').text = str(1)

    # Dense offset settings
    ET.SubElement(tinsar, 'property', name='do dense offsets').text = str(True)
    ET.SubElement(tinsar, 'property', name='Ampcor window width').text = str(64)
    ET.SubElement(tinsar, 'property', name='Ampcor window height').text = str(64)
    ET.SubElement(tinsar, 'property', name='Ampcor search window width').text = str(20)
    ET.SubElement(tinsar, 'property', name='Ampcor search window height').text = str(20)
    ET.SubElement(tinsar, 'property', name='Ampcor skip width').text = str(32)
    ET.SubElement(tinsar, 'property', name='Ampcor skip height').text = str(32)
    ET.SubElement(tinsar, 'property', name='Ampcor margin').text = str(50)
    ET.SubElement(tinsar, 'property', name='Ampcor oversampling factor').text = str(32)
    ET.SubElement(tinsar, 'property', name='Range shift').text = str(0)
    ET.SubElement(tinsar, 'property', name='Azimuth shift').text = str(0)
    ET.SubElement(tinsar, 'property', name='SNR Threshold factor').text = str(8.0)

    # Water masking
    # ET.SubElement(tinsar, 'property', name='apply water mask').text = str(True)
    # ET.SubElement(tinsar, 'property', name='water mask file name').text = '/path/to/water_mask.wbd'

    # Ionospheric correction
    if do_iono == 'True':
        ET.SubElement(tinsar, 'property', name='do ionosphere correction').text = do_iono
        ET.SubElement(tinsar, 'property', name='apply ionosphere correction').text = do_iono
        ET.SubElement(tinsar, 'property', name='start ionosphere step').text = 'subband'
        ET.SubElement(tinsar, 'property', name='end ionosphere step').text = 'esd'
        ET.SubElement(tinsar, 'property', name='height of ionosphere layer in km').text = str(200.0)
        ET.SubElement(tinsar, 'property', name='apply polynomial fit before filtering ionosphere phase').text = str(True)
        ET.SubElement(tinsar, 'property', name='maximum window size for filtering ionosphere phase').text = str(200)
        ET.SubElement(tinsar, 'property', name='minimum window size for filtering ionosphere phase').text = str(100)
        ET.SubElement(tinsar, 'property', name='maximum window size for filtering ionosphere azimuth shift').text = str(150)
        ET.SubElement(tinsar, 'property', name='minimum window size for filtering ionosphere azimuth shift').text = str(75)
        ET.SubElement(tinsar, 'property', name='correct phase error caused by ionosphere azimuth shift').text = str(2)

    # Save XML
    tree = ET.ElementTree(data)
    tree.write(os.path.join(xmldirectories[0], xmlname))

In [6]:
# took these from UNAVCO for plotting the results from topsApp

# Utility to plot a 2D array
def plotdata(GDALfilename, band=1,
             title=None,colormap='gray',
             aspect=1, background=None,
             datamin=None, datamax=None,
             interpolation='nearest',
             nodata = None,
             draw_colorbar=True, colorbar_orientation="horizontal"):
    
    # Read the data into an array
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    data = ds.GetRasterBand(band).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    
    try:
        if nodata is not None:
            data[data == nodata] = np.nan
    except:
        pass
        
    # getting the min max of the axes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+data.shape[1]*deltax
    lasty = firsty+data.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])

    # put all zero values to nan and do not plot nan
    if background is None:
        try:
            data[data==0]=np.nan
        except:
            pass
    
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax,
                    cmap=colormap, extent=[xmin,xmax,ymin,ymax],
                    interpolation=interpolation)
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show()
    
    # clearing the data
    data = None

# Utility to plot interferograms
def plotcomplexdata(GDALfilename,
                    title=None, aspect=1,
                    datamin=None, datamax=None,
                    interpolation='nearest',
                    draw_colorbar=None, colorbar_orientation="horizontal"):
    # Load the data into numpy array
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    slc = ds.GetRasterBand(1).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    
    # getting the min max of the axes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+slc.shape[1]*deltax
    lasty = firsty+slc.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])

    # put all zero values to nan and do not plot nan
    try:
        slc[slc==0]=np.nan
    except:
        pass

    
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(slc), vmin = datamin, vmax=datamax,
                   cmap='gray', extent=[xmin,xmax,ymin,ymax],
                   interpolation=interpolation)
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(slc), cmap='rainbow',
                    vmin=-np.pi, vmax=np.pi,
                    extent=[xmin,xmax,ymin,ymax],
                    interpolation=interpolation)
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2, orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show()
    
    # clearing the data
    slc = None

# Utility to plot multiple similar arrays
def plotstackdata(GDALfilename_wildcard, band=1,
                  title=None, colormap='gray',
                  aspect=1, datamin=None, datamax=None,
                  interpolation='nearest',
                  draw_colorbar=True, colorbar_orientation="horizontal"):
    # get a list of all files matching the filename wildcard criteria
    GDALfilenames = glob.glob(GDALfilename_wildcard)
    
    # initialize empty numpy array
    data = None
    for GDALfilename in GDALfilenames:
        ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
        data_temp = ds.GetRasterBand(band).ReadAsArray()   
        ds = None
        
        if data is None:
            data = data_temp
        else:
            data = np.vstack((data,data_temp))

    # put all zero values to nan and do not plot nan
    try:
        data[data==0]=np.nan
    except:
        pass            
            
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax,
                    cmap=colormap, interpolation=interpolation)
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show() 

    # clearing the data
    data = None

# Utility to plot multiple simple complex arrays
def plotstackcomplexdata(GDALfilename_wildcard,
                         title=None, aspect=1,
                         datamin=None, datamax=None,
                         interpolation='nearest',
                         draw_colorbar=True, colorbar_orientation="horizontal"):
    # get a list of all files matching the filename wildcard criteria
    GDALfilenames = glob.glob(GDALfilename_wildcard)
    print(GDALfilenames)
    # initialize empty numpy array
    data = None
    for GDALfilename in GDALfilenames:
        ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
        data_temp = ds.GetRasterBand(1).ReadAsArray()
        ds = None
        
        if data is None:
            data = data_temp
        else:
            data = np.vstack((data,data_temp))

    # put all zero values to nan and do not plot nan
    try:
        data[data==0]=np.nan
    except:
        pass              
            
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(data), vmin=datamin, vmax=datamax,
                   cmap='gray', interpolation='nearest')
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(data), cmap='rainbow',
                            interpolation='nearest')
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2,orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show() 
    
    # clearing the data
    data = None

# Establish working directroy and data paths

In [10]:
# Establish working directories, and locate Sentinel-1 SLC .zip files
proj_name = 'SabineRS'
work_dir, ifdirectories, tsdirectories, xmldirectories = project_dir(proj_name, 'linux')

Creating Project Working Directory in your "~" directory


In [11]:
# assuming you have downloaded .zip files covering your AOI from ASF Vertex
# enter the file directory below
slc_zips = '/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93'

slc_zips_list = sorted(os.listdir(slc_zips), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
slc_zips_dirs = [os.path.join(slc_zips, slc) for slc in slc_zips_list]
slc_zips_dates = [slc[17:25] for slc in slc_zips_list]

# Get bbox drawn in 01_get_slc.ipynb

In [14]:
isce_aoi_file = open('isceaoi.txt', 'r+')
aoi = isce_aoi_file.read()
isce_aoi = [float(num) for num in re.findall(r'[-+]?\d*\.\d+', aoi)]

# Retrieve water mask needed for topsApp
- use it just before unwrapping

In [ ]:
os.chdir(work_dir)

snaphuexport_cmd = [
    '/home/wcc/tools/miniforge/envs/isce/lib/python3.8/site-packages/isce/applications/wbd.py',
    isce_aoi[0],
    isce_aoi[1],
    isce_aoi[2],
    isce_aoi[3]
]

wbd_file = os.path.expandvars(os.path.join(work_dir,'*.wbd'))

# Run the command
try:
    result = subprocess.run(snaphuexport_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print("Command executed successfully!")
except subprocess.CalledProcessError as e:
    print(f"Command failed with error code {e.returncode}")
    print(f"Error message: {e.stderr}")

# Get s-1 orbit files (.EOF) using sentineleof library

In [50]:
#below downloads orbit files for your S1 SLC imagery to orbits_dir created with project_dir
for i,slc in enumerate(slc_zips_list):
    eof.download.download_eofs(
        orbit_dts=slc_zips_dates[i],  # slc date in str YYYYMMDD format
        missions=['S1A', 'S1B'],        # gets both S1 missions, third was just launched (2024) so may need updating
        sentinel_file=slc,              # image name
        save_dir=ifdirectories[2],      # orbits_dir
        orbit_type='precise'            # can be 'precise' or 'restituted'
    )

# Create Sentinel-1 SLC .xml files
- One reference and one secondary for each image

In [15]:
ref_sec_xml(slc_zips_list, slc_zips_dirs)

# Create topsApp.xml file
- look into computing baselines, running dense offsets for wrapped and unwrapped ifgs, water masking, shadow masking, and ionosphere correction
- one for each interferogram in each triplet

in order of importance
1. water mask in radar and for mintpy (wbd.py)
2. add in all parameters for topsApp (dense offsets etc.)
3. compute baselines
4.  automation for all SLCs (including removing old data and organizing to mintpy standards)
5. shadow masking

In [41]:
# stores these in xmldirectories[0]

do_iono = 'True'

for i, m in enumerate(slc_zips_list):
    if i < len(slc_zips_list) - 4:
        for date in [f'{m[17:25]}_{slc_zips_list[i+1][17:25]}', f'{m[17:25]}_{slc_zips_list[i+2][17:25]}', f'{m[17:25]}_{slc_zips_list[i+3][17:25]}']:
            topsApp_xml(f'{date}topsApp.xml', do_iono)

    elif i == len(slc_zips_list) - 3:
        for date in [f'{m[17:25]}_{slc_zips_list[i+1][17:25]}', f'{m[17:25]}_{slc_zips_list[i+2][17:25]}']:
            topsApp_xml(f'{date}topsApp.xml', do_iono)


    elif i == len(slc_zips_list) - 2:
        topsApp_xml(f"{m[17:25]}_{slc_zips_list[i+1][17:25]}topsApp.xml", do_iono)

    elif i == len(slc_zips_list) - 1:
        break

# topsApp interferometry (finally!)
- Need to split interferogram processing into three steps
1. preprocess to filter
2. apply water mask
3. unwrap to endup

In [42]:
# should import isce toolbox and print the version of it here
# if nothing prints or nothing imports, you need to reinstall isce2

import isce
from topsApp import TopsInSAR
isce.version.release_version

'2.6.3'

In [43]:
# this is used to change directory into the storage directory for all interferograms
# should be used, else will save results to github repo
os.chdir(ifdirectories[3])
print(ifdirectories[3])

/home/wcc/SabineRS_InSAR/interferometry/work


In [ ]:
xml_path = os.path.join(xmldirectories[0] , sorted(os.listdir(xmldirectories[0]), key=lambda x: datetime.strptime(x[:8], '%Y%m%d'))[0])
insar = TopsInSAR(name="topsApp", cmdline=xml_path)
insar.configure()
insar.run()


# for xml in sorted(os.listdir(xmldirectories[0]), key=lambda x: datetime.strptime(x[:8], '%Y%m%d'))[:3]:
#     xml_path = os.path.join(xmldirectories[0] , xml)
#     insar = TopsInSAR(name="topsApp", cmdline=xml_path)
#     insar.configure()
#     insar.run()

2024-12-09 17:41:05,654 - isce.insar - INFO - ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 
None
The currently supported sensors are:  ['SENTINEL1']
Input XML files:  ['/vsizip//home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.zip/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.SAFE/annotation/s1a-iw3-slc-vv-20191001t001839-20191001t001904-029258-035306-006.xml']
Input TIFF files:  ['/vsizip//home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.zip/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.SAFE/measurement/s1a-iw3-slc-vv-20191001t001839-20191001t001904-029258-035306-006.tiff']
Manifest files:  ['/vsizip//home/wcc/Desktop/SabineRS/Sentinel-1

processing line:   1508 of   1508



file width: 24568, file length: 1508

inputfile byte order: little endian
input file image offset [byte]: 0
input file line offset [byte]: 196544


Chi squared: 0.006359
Polynomial Order: 5 - by - 3 
11890.4	-476.23	239.863	7.18898	
-47309.2	2127.49	-530.205	-9.38084	
47058.2	-2346.31	100.596	3.57171	
-2.71035e-09	3.09008e-09	-1.07324e-09	0	
1.8131e-09	-8.82659e-10	0	0	
-4.32332e-10	0	0	0	
Misfit radians - Max: 0.01770277365721995 , Min : -0.009258800460884231 
2024-12-09 17:49:22,787 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_LINES has been set to the default value 1508 which is the number of lines in the slc image.
2024-12-09 17:49:22,788 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_WIDTH has been set to the default value 24568 which is the width of the slc image.
2024-12-09 17:49:22,788 - isce.stdproc.resamp_slc - WARNING - No flattening requested


processing line:   1508 of   1508



...

No Range Carrier provided.
Assuming zero range carrier.
GDAL open (R): ion/lower/fine_interferogram/IW3/secondary_burst_09.slc.vrt
API open (WR): ion/lower/fine_interferogram/IW3/secondary_resamp_burst_09.slc
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 5 - by - 3 
11982.1	-627.567	14.5483	0.804826	
-47485.3	2428.09	-79.6053	-3.35543	
47046.2	-2347.22	100.67	3.5756	
-2.71567e-09	2.96726e-09	-1.04929e-09	0	
1.86878e-09	-8.62092e-10	0	0	
-4.64324e-10	0	0	0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
GDAL open (R): fine_offsets/IW3/range_09.off.vrt
GDAL open (R): fine_offsets/IW3/azimuth_09.off.vrt
Polynomial Order: 0 - by - 2 
0.270431	-7.41717	-121306	
  
  << Resample one image to another image coordinates >>
  
 Input Image Dimensions: 
       24568  pixels
        1508 lines
  
 Output Image Dimensions: 
       24569 pixels
        1508 lines
   
 Number of threads:            8
 Complex data interpolation
 Initializing Si

line: 12224


number of lines written to file: 12240
Rescaling magnitude


line: 12239

GDAL close: merged/topophase.flat.vrt
API close:  merged/filt_topophase.flat
GDAL open (R): merged/filt_topophase.flat.vrt
API open (WR): merged/phsig.cor

 << PS filtering >>


 interferogram width:  4913  number of lines/patch:  3700
 start line:        1  number of lines:   12240
 start sample:      1  end sample:         4913
 wisdomFile, length =            0

 azimuth buffer size:                             3700
 overlap between azimuth patches:                  200
 total overlap between azimuth patches:            264
 offset in overlap region for phase  bootstrap:    132
 lines to increment for the next patch:           3436
 number of patches:                                  4

 PATCH:   1   starting line:     0    lines read: 3700
 starting output line:    1   ending output line: 3568

 PATCH:   2   starting line:  3436    lines read: 3700
 starting output line:  133   ending output line: 3568

 PATCH:   3   starting line:  6872    lines read: 3700
 starting output line:  

In [ ]:
# step processing here
#do preprocess to filter


In [ ]:
# use wbd.py or waterMask.py to retrieve and download needed watermask
# apply watermask (using isce2, or manually with gdal and rasterio)


In [ ]:
# continue with InSAR using the water masked data for unwrapping
# do unwrap to endup


In [ ]:
# can run these steps separately if you want more info on where process may be failing

# insar = TopsInSAR(name="topsApp", cmdline=xml_path)
# insar.configure()
# insar.startup()
# insar.runComputeBaseline()
# insar.verifyDEM()
# insar.verifyGeocodeDEM()
# insar.runTopo()
# insar.runSubsetOverlaps()
# insar.runCoarseOffsets()
# insar.runCoarseResamp()
# insar.runOverlapIfg()
# insar.runPrepESD()
# insar.runESD()
# insar.runRangeCoreg()
# insar.runFineOffsets()
# insar.runFineResamp()
# insar.runIon()
# insar.runBurstIfg()
# insar.runMergeBursts()
# #add dense offsets here for wrapped phase if you'd like
# insar.runFilter()
# insar.runUnwrapper()    # can also run 2stage unwrapper and dump to pickle if you want
# insar.runGeocode(insar.geocode_list, insar.do_unwrap, insar.geocode_bbox)
# insar.runDenseOffsets()
# insar.runOffsetFilter()
# insar.runGeocode(insar.off_geocode_list, True, insar.geocode_bbox, True)
# insar.endup()

Steps include:
- runPreprocessor
- runComputeBaseline
- verifyDEM
- verifyGeocodeDEM
- runTopo
- runSubsetOverlaps
- runCoarseOffsets
- runCoarseResamp
- runOverlapIfg
- runPrepESD
- runESD
- runRangeCoreg
- runFineOffsets
- runFineResamp
- runIon
- runBurstIfg
- runMergeBursts
- runFilter
- runGeocode
- runDenseOffsets
- runOffsetFilter

# Generate baseline file for interferograms
- uses isce2/contrib/stack/topsStack/computeBaseline.py, you will need to provide your path

# Arrange file directories for use in MintPy
- https://github.com/insarlab/MintPy-tutorial/blob/main/workflows/smallbaselineApp.ipynb
- reference the Fernandina example set
- for 'reference' and 'merged/reference' files, only needed from first ifg
- remove files not needed in mintpy to save storage space

NEED TO FIGURE THIS OUT FIRST, AUTOMATION ALREADY WORKS BUT OVERWRITES ITSELF

# Generate smallbaselineApp files needed for MintPy time series (might have to run this in mintpy environment)
- need prep_isce and load_data from MintPy
- https://mintpy.readthedocs.io/en/stable/dir_structure/

In [ ]:
# mintpy needs

# mintpy.load.processor        = isce
# ##---------for ISCE only:
# mintpy.load.metaFile         = $DATA_DIR/GalapagosSenDT128/reference/IW*.xml
# mintpy.load.baselineDir      = $DATA_DIR/GalapagosSenDT128/baselines
# ##---------interferogram datasets:
# mintpy.load.unwFile          = $DATA_DIR/GalapagosSenDT128/merged/interferograms/*/filt_*.unw
# mintpy.load.corFile          = $DATA_DIR/GalapagosSenDT128/merged/interferograms/*/filt_*.cor
# mintpy.load.connCompFile     = $DATA_DIR/GalapagosSenDT128/merged/interferograms/*/filt_*.unw.conncomp
# ##---------geometry datasets:
# mintpy.load.demFile          = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/hgt.rdr
# mintpy.load.lookupYFile      = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/lat.rdr
# mintpy.load.lookupXFile      = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/lon.rdr
# mintpy.load.incAngleFile     = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/los.rdr
# mintpy.load.azAngleFile      = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/los.rdr
# mintpy.load.shadowMaskFile   = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/shadowMask.rdr

